In [1]:
ROOT = './dataset'

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]= '5'
import torch

import sys
sys.path.append('/mnt/mhjc/RS/ogblsc/ogb')

In [3]:
from ogb.lsc import PCQM4MDataset
dataset = PCQM4MDataset(only_smiles = True)

# get i-th molecule and its target value (nan for test data)
i = 1234
print(dataset[i]) # ('O=C1C=CC(O1)C(c1ccccc1C)O', 5.292614392225)

Using backend: pytorch


('O=C1C=CC(O1)C(c1ccccc1C)O', 5.292614392225)


In [ ]:
from ogb.utils import smiles2graph

# smiles2graph takes a SMILES string as input and returns a graph object
# requires rdkit to be installed.
# You can write your own smiles2graph
graph_obj = smiles2graph('O=C1C=CC(O1)C(c1ccccc1C)O')

# convert each SMILES string into a molecular graph object by calling smiles2graph
# This takes a while (a few hours) for the first run
dataset = PCQM4MDataset(smiles2graph = smiles2graph)

# get i-th molecule and its target value (nan for test data)
i = 1234
print(dataset[i]) # (graph_obj, 5.292614392225)

In [4]:
split_dict = dataset.get_idx_split()
train_idx = split_dict['train'] # numpy array storing indices of training paper nodes
valid_idx = split_dict['valid'] # numpy array storing indices of validation paper nodes
test_idx = split_dict['test'] # numpy array storing indices of testing paper nodes

import os.path as osp
from dgl.data.utils import load_graphs, save_graphs, Subset
folder = 'dataset/pcqm4m_kddcup2021'
processed_dir = osp.join(folder, 'processed')
pre_processed_file_path = osp.join(processed_dir, 'dgl_data_processed')
graphs, label_dict = load_graphs(pre_processed_file_path)

graphs

label_dict

In [9]:
from ogb.utils import smiles2graph

# if you use DGL (requires dgl to be installed) 
from ogb.lsc import DglPCQM4MDataset
dgl_dataset = DglPCQM4MDataset(root = ROOT, smiles2graph = smiles2graph) # load all of graphs list

# dgl_dataset2 = DglPCQM4MDataset(root = ROOT, smiles2graph = smiles2graph, idx_list = [ int(i) for i in train_idx] ) # load subset of graphs list => 전체 load 하는것과 비슷하게 오래걸린다..

dgl_dataset[100][0].ndata['feat'].shape

from ogb.graphproppred.mol_encoder import AtomEncoder, BondEncoder
atom_encoder = AtomEncoder(emb_dim = 100) # Pytorch Module class w/ learnable parameters
bond_encoder = BondEncoder(emb_dim = 100) # Pytorch Module class w/ learnable parameters

atom_emb = atom_encoder(dgl_dataset[0][0].ndata['feat']) # node_feat is input atom feature in Pytorch Tensor
edge_emb = bond_encoder(dgl_dataset[0][0].edata['feat']) # edge_feat is input edge feature in Pytorch Tensor

In [10]:
len(dgl_dataset)

3803453

In [12]:
from dgl.dataloading import GraphDataLoader
dataloader = GraphDataLoader(
    dgl_dataset,
    batch_size=1024*16,
    drop_last=False,
    shuffle=True)

batched_graph, labels = dataloader.__iter__().__next__()

transferer = dgl.dataloading.AsyncTransferer(torch.device('cuda:1'))

from ogb.graphproppred.mol_encoder import AtomEncoder, BondEncoder
atom_encoder = AtomEncoder(emb_dim = 100).to('cuda:1')

feats = batched_graph.ndata['feat']
feats = atom_encoder(feats)

model = Regressor(in_dim = 100, 
                  hidden_dim = 24).to('cuda:1')

logits = model(batched_graph.to('cuda:1'), feats)

loss = F.mse_loss(logits, labels.to('cuda:1'))

logits.shape

labels.to('cuda:1').shape

loss

In [15]:
import torch.nn.functional as F
from sklearn.metrics import roc_auc_score
from ogb.graphproppred.mol_encoder import AtomEncoder, BondEncoder
import dgl

def train(dataloader, model, atom_encoder = AtomEncoder(emb_dim = 100),
    bond_encoder = BondEncoder(emb_dim = 100), max_iter = 5, verbose = 1, lr = 1e-3):
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    transferer = dgl.dataloading.AsyncTransferer(torch.device('cuda:0'))
   
    for e in range(max_iter):
        count = 0
        for batched_graph, labels in dataloader:
            feats = batched_graph.ndata['feat']
#             feats = atom_encoder(feats) #.to('cuda:0')
            feats_gpu = transferer.async_copy( feats, torch.device('cuda:0'))
#             batched_graph_gpu = transferer.async_copy( batched_graph, torch.device('cuda:0'))
            labels_gpu = transferer.async_copy( labels.reshape([-1,1]), torch.device('cuda:0'))

            # Forward
            logits = model(batched_graph.to('cuda:0'), feats_gpu.wait()) 
            
            # Compute loss
            loss = F.mse_loss(logits, labels_gpu.wait()) #.to('cuda:0')
            if count % 32 == 0:
                print('In count {}, loss: {:.3f}'.format(count, loss) )
            count += 1
            
            # Backward
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()  

        if e % verbose == 0:
            print('In epoch {}, loss: {:.3f}'.format(e, loss))
                   

async_copy [doc link](https://docs.dgl.ai/_modules/dgl/dataloading/async_transferer.html#AsyncTransferer)
AsyncTransferer [doc link](https://docs.dgl.ai/api/python/dgl.dataloading.html?highlight=asynctransferer#dgl.dataloading.AsyncTransferer)

GraphCollator [source link](https://docs.dgl.ai/_modules/dgl/dataloading/dataloader.html#GraphCollator)
GraphDataLoader [source link](https://docs.dgl.ai/_modules/dgl/dataloading/pytorch.html#GraphDataLoader)


In [14]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import init
import dgl.nn.pytorch as dglnn
import dgl

class Regressor(nn.Module):
    def __init__(self, in_dim, hidden_dim):
        super(Regressor, self).__init__()
        self.conv1 = dglnn.GraphConv(in_dim, hidden_dim, allow_zero_in_degree = True)
        self.conv2 = dglnn.GraphConv(hidden_dim, hidden_dim, allow_zero_in_degree = True)
        self.regress = nn.Linear(hidden_dim, 1)
        self.atom_encoder = AtomEncoder(emb_dim = in_dim).to(torch.device('cuda:0'))

    def forward(self, g, h):
        # Apply graph convolution and activation.        
        h = self.atom_encoder(h)
        h = F.relu(self.conv1(g, h))
        h = F.relu(self.conv2(g, h))
        with g.local_scope():
            g.ndata['h'] = h
            # Calculate graph representation by average readout.
            hg = dgl.mean_nodes(g, 'h')
            return self.regress(hg)

In [17]:
del model
# del dataloader
torch.cuda.empty_cache()
import gc
gc.collect()

56

In [16]:
from ogb.graphproppred.mol_encoder import AtomEncoder, BondEncoder

model = Regressor(in_dim = 50, 
                  hidden_dim = 24).to('cuda:0')
train(dataloader, model, atom_encoder = AtomEncoder(emb_dim = 50)) #.to('cuda:0')

In count 0, loss: nan
In count 32, loss: nan


KeyboardInterrupt: 

In [74]:
atom_encoder = AtomEncoder(emb_dim = 50)
graph = dgl_dataset[0][0]
feats = atom_encoder(graph.ndata['feat'])
model(graph.to('cuda:0'), feats.to('cuda:0') )

DGLError: [06:01:15] /opt/dgl/src/runtime/cuda/cuda_device_api.cc:196: Check failed: e == cudaSuccess || e == cudaErrorCudartUnloading: CUDA: an illegal memory access was encountered
Stack trace:
  [bt] (0) /mnt/mhjc/miniconda3/envs/ogblsc/lib/python3.7/site-packages/dgl/libdgl.so(dmlc::LogMessageFatal::~LogMessageFatal()+0x4f) [0x7f0979b9a01f]
  [bt] (1) /mnt/mhjc/miniconda3/envs/ogblsc/lib/python3.7/site-packages/dgl/libdgl.so(dgl::runtime::CUDADeviceAPI::CopyDataFromTo(void const*, unsigned long, void*, unsigned long, unsigned long, DLContext, DLContext, DLDataType, void*)+0x7b) [0x7f097a3feaeb]
  [bt] (2) /mnt/mhjc/miniconda3/envs/ogblsc/lib/python3.7/site-packages/dgl/libdgl.so(dgl::runtime::NDArray::CopyFromTo(DLTensor*, DLTensor*, void*)+0x267) [0x7f097a2ba647]
  [bt] (3) /mnt/mhjc/miniconda3/envs/ogblsc/lib/python3.7/site-packages/dgl/libdgl.so(dgl::runtime::NDArray::CopyTo(DLContext const&) const+0xee) [0x7f097a2f158e]
  [bt] (4) /mnt/mhjc/miniconda3/envs/ogblsc/lib/python3.7/

In [61]:
print(dgl_dataset[0])

(Graph(num_nodes=18, num_edges=40,
      ndata_schemes={'feat': Scheme(shape=(9,), dtype=torch.int64)}
      edata_schemes={'feat': Scheme(shape=(3,), dtype=torch.int64)}), tensor(3.0477))


In [63]:
print(dataset[0])

('Cc1ccc(cc1)C1C=c2cnccc2=NC1=O', 3.0476751256)
